## Get GPS history from .txt file

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
from PIL import Image
import math

from dotenv import load_dotenv
import os

In [28]:
#directories of kitti lidar dataset
testpath = 'testing/oxts/'
trainpath = 'training/oxts/'

In [3]:
def get_file_name(x, scenario='um_', ext='.txt'):
    #generate file name string
    #in the style of kitti file names
    mlen = 6
    instr = str(x)
    zlen = mlen - len(instr)
    numstr = ''

    for i in range(zlen):
        numstr += '0'
    numstr += instr
    return scenario+numstr+ext

In [4]:
def get_gps(x):
    #returns lat, long, heading of specific timestep
    with open(trainpath+get_file_name(x, scenario='uu_')) as f:
        text = f.read()
    list = text.split(' ')
    f.close()
    lat = list[0]
    long = list[1]
    heading = list[5]
    return [float(lat), float(long), float(heading)]

In [5]:
def make_gps_list(last):
    #makes list of gps history
    #input is an int indicating last file id
    coords = []
    for i in range(last+1):
        point = get_gps(i)
        coords.append(point)
    coords = np.asarray(coords)
    return coords

In [27]:
#list coordinates and heading
coords = make_gps_list(10)
coords

array([[49.00929592,  8.43740956, -0.26755467],
       [49.00924383,  8.4376788 , -0.26708267],
       [49.00918874,  8.43794475, -0.28912767],
       [49.00913695,  8.43821767, -0.25900867],
       [49.00908648,  8.43849399, -0.26364167],
       [49.00903671,  8.43876618, -0.26486367],
       [49.00898048,  8.43976423,  1.22561633],
       [49.00915782,  8.43985119,  1.24870033],
       [49.0093346 ,  8.43994865,  1.19607233],
       [49.00950766,  8.44005921,  1.14287233],
       [49.02582351,  8.44841156,  2.68752133]])

## Access API and download images

In [21]:
#load google api key, set download path
load_dotenv()
api_key = os.getenv('API_KEY')
dlpath = 'download/'

In [22]:
def make_url(lat,long,zoom=19,key=api_key):
    #note: google maps static api ignores gps precision past 6th decimal place
    url = "https://maps.googleapis.com/maps/api/staticmap?center={},{}&zoom={}&size=800x800&style=feature:road|element:geometry|color:0x000000&style=feature:road.arterial|element:all|weight:0&style=feature:landscape|element:all|color:0xffffff&style=feature:all|element:labels|visibility:off&style=feature:water|visibility:off&style=feature:poi|element:all|visibility:off&key={}".format(lat,long,zoom,key)
    return url

In [23]:
def access_api(url,n,spath=dlpath):
    fname = get_file_name(n,scenario='uu_',ext='.png')
    with urllib.request.urlopen(url) as page:
        with open(spath+fname, 'wb') as f:
            f.write(page.read())
        f.close()

In [24]:
def download_tiles(coords):
    #given a list of gps coordinates
    #download corresponding tiles from api
    for count, c in enumerate(coords):
        url = make_url(c[0],c[1])
        access_api(url,count)
    print('done!')

In [25]:
download_tiles(coords)
#make_url(49.0177435,8.44004242)
#get_file_name(0,scenario='umm_',ext='.png')

done!


## Crop and rotate to desired heading

In [26]:
BOX_W = 20
BOX_L_F = 46
BOX_L_R = 6
finalpath = 'cropped/'

def r2d(theta):
    return (theta*180)/math.pi

def d2r(theta):
    return (theta*math.pi)/180

def calc_mpp(lat, zoom=20):
    #calculates meters per pixel at a certain latitude
    return 156543.03392 * math.cos(d2r(lat)) / (2**20)

#calc_mpp(49) returns 0.0979, assume constant in the same city

def rotate_crop_image(x,heading,path=dlpath,spath=finalpath):
    #given a file number, rotate and crop the image
    #heading in degrees
    imgname = get_file_name(x,scenario='uu_', ext='.png')
    fpath = path + imgname
    img = Image.open(fpath)
    img = img.rotate(-heading+90)

    #find coordinates for middle of picture
    xmax = img.size[0]
    ymax = img.size[1]
    xmid = round(xmax/2)
    ymid = round(ymax/2)
    
    #find crop size in pixels (upper left, lower right)
    halfwidth = 100
    rearlength = 60
    frontlength = 460
    croptuple = (xmid-halfwidth,
                ymid-frontlength,
                xmid+halfwidth,
                ymid+rearlength)
    
    img = img.crop(croptuple)
    #img.show()
    img.save(spath+imgname)
    


In [55]:
for count, point in enumerate(coords[:,2]):
    hd = r2d(point)
    rotate_crop_image(count,hd)